In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pymongo  
from pymongo import MongoClient
import pyodbc
from datetime import datetime
import time
import MongoConnect
import pandas as pd
import importlib
from selenium import webdriver
import os
import youtube_dl 
import preprocessing as cp

In [3]:
song_train=pd.read_csv('../data/song_train.csv',encoding="ISO-8859-1")

In [4]:
song_train.head()

,Unnamed: 0,_id,artist,chord,error,rating,tab_href,tab_title,version,web_html
0,0,5c42e9649fc5af27c4b76a1f,The Beatles,"['G', 'Bm', 'Em', 'Em7', 'D', 'B', 'Em', 'Cmaj...",0,74,https://tabs.ultimate-guitar.com/tab/the_beatl...,A Day In The Life,Ver 1,"<!DOCTYPE html><html xmlns=""http://www.w3.org/..."
1,1,5c42e96f9fc5af27c4b76a21,The Beatles,"['G', 'Bm', 'Em', 'C', 'Em', 'Cmaj', 'G', 'Bm'...",0,22,https://tabs.ultimate-guitar.com/tab/the_beatl...,A Day In The Life,Ver 2,"<!DOCTYPE html><html xmlns=""http://www.w3.org/..."
2,2,5c42e9799fc5af27c4b76a23,The Beatles,"['Em7', 'C/B', 'Asus2', 'Dsus2', 'B7sus4', 'G'...",0,"1,450",https://tabs.ultimate-guitar.com/tab/the_beatl...,A Day In The Life,Ver 3,"<!DOCTYPE html><html xmlns=""http://www.w3.org/..."
3,3,5c42e9839fc5af27c4b76a25,The Beatles,"['G', 'Bm', 'Em', 'Em7', 'C', 'C6', 'C', 'C', ...",0,74,https://tabs.ultimate-guitar.com/tab/the_beatl...,A Day In The Life,Ver 4,"<!DOCTYPE html><html xmlns=""http://www.w3.org/..."
4,4,5c42eaba9fc5af27c4b76a2d,The Beatles,"['G', 'Bm', 'Em', 'Em7', 'C', 'C', 'G', 'F', '...",0,4,https://tabs.ultimate-guitar.com/tab/the_beatl...,A Day In The Life,Ver 5,"<!DOCTYPE html><html xmlns=""http://www.w3.org/..."


In [65]:
#取出每組裡最大rating的譜
song_train_group=song_train.sort_values('rating', ascending=True).groupby('tab_title', as_index=False).first()
song_train_group

,tab_title,Unnamed: 0,_id,artist,chord,error,rating,tab_href,version,web_html
0,A Day In The Life,2,5c42e9799fc5af27c4b76a23,The Beatles,"['Em7', 'C/B', 'Asus2', 'Dsus2', 'B7sus4', 'G'...",0,"1,450",https://tabs.ultimate-guitar.com/tab/the_beatl...,Ver 3,"<!DOCTYPE html><html xmlns=""http://www.w3.org/..."
1,A Hard Days Night,9,5c42eaf79fc5af27c4b76a39,The Beatles,"['G', 'C', 'G', 'F', 'G', 'C', 'G', 'F', 'G', ...",0,12,https://tabs.ultimate-guitar.com/tab/the_beatl...,Ver 4,"<!DOCTYPE html><html xmlns=""http://www.w3.org/..."
2,A Shot Of Rhythm And Blues,15,5c42eb349fc5af27c4b76a45,The Beatles,"['D', 'D', 'G7', 'D', 'A7', 'G7', 'A7', 'G7', ...",0,3,https://tabs.ultimate-guitar.com/tab/the_beatl...,Ver 1,"<!DOCTYPE html><html xmlns=""http://www.w3.org/..."
3,A Taste Of Honey,17,5c42eb499fc5af27c4b76a49,The Beatles,"['F#m', 'A', 'E', 'F#m', 'F#m', 'Faug', 'A', '...",0,24,https://tabs.ultimate-guitar.com/tab/the_beatl...,Ver 1,"<!DOCTYPE html><html xmlns=""http://www.w3.org/..."
4,Across The Universe,19,5c42eb5d9fc5af27c4b76a4d,The Beatles,"['D', 'Bm', 'F#m', 'Em7', 'A', 'A7', 'D', 'Bm'...",0,"1,460",https://tabs.ultimate-guitar.com/tab/the_beatl...,Ver 1,"<!DOCTYPE html><html xmlns=""http://www.w3.org/..."
5,Act Naturally,24,5c42eb909fc5af27c4b76a57,The Beatles,"['D7', 'G', 'G', 'C', 'G', 'D7', 'G', 'C', 'D7...",0,186,https://tabs.ultimate-guitar.com/tab/the_beatl...,Ver 1,"<!DOCTYPE html><html xmlns=""http://www.w3.org/..."
6,Aint She Sweet,27,5c42ebae9fc5af27c4b76a5d,The Beatles,"['E', 'Edim', 'B', 'G#7', 'C#7', 'F#7', 'B7', ...",0,32,https://tabs.ultimate-guitar.com/tab/the_beatl...,Ver 1,"<!DOCTYPE html><html xmlns=""http://www.w3.org/..."
7,All Ive Got To Do,32,5c42ebe19fc5af27c4b76a67,The Beatles,"['E11', 'C#m', 'E', 'F#m', 'Am', 'A', 'E11', '...",0,10,https://tabs.ultimate-guitar.com/tab/the_beatl...,Ver 4,"<!DOCTYPE html><html xmlns=""http://www.w3.org/..."
8,All My Loving,34,5c42ebf69fc5af27c4b76a6b,The Beatles,"['Em', 'A7', 'D', 'Bm', 'G', 'Em', 'C', 'A7', ...",0,"2,280",https://tabs.ultimate-guitar.com/tab/the_beatl...,Ver 1,"<!DOCTYPE html><html xmlns=""http://www.w3.org/..."
9,All Together Now,38,5c42ec1e9fc5af27c4b76a73,The Beatles,"['G', 'D7', 'G', 'D7', 'G', 'A', 'B', 'C', 'D'...",0,11,https://tabs.ultimate-guitar.com/tab/the_beatl...,Ver 2,"<!DOCTYPE html><html xmlns=""http://www.w3.org/..."


In [11]:
song_train['song_name']=song_train.tab_title+song_train.artist
song_train.head()

,Unnamed: 0,_id,artist,chord,error,rating,tab_href,tab_title,version,web_html,song_name
0,0,5c42e9649fc5af27c4b76a1f,The Beatles,"['G', 'Bm', 'Em', 'Em7', 'D', 'B', 'Em', 'Cmaj...",0,74,https://tabs.ultimate-guitar.com/tab/the_beatl...,A Day In The Life,Ver 1,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",A Day In The Life The Beatles
1,1,5c42e96f9fc5af27c4b76a21,The Beatles,"['G', 'Bm', 'Em', 'C', 'Em', 'Cmaj', 'G', 'Bm'...",0,22,https://tabs.ultimate-guitar.com/tab/the_beatl...,A Day In The Life,Ver 2,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",A Day In The Life The Beatles
2,2,5c42e9799fc5af27c4b76a23,The Beatles,"['Em7', 'C/B', 'Asus2', 'Dsus2', 'B7sus4', 'G'...",0,"1,450",https://tabs.ultimate-guitar.com/tab/the_beatl...,A Day In The Life,Ver 3,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",A Day In The Life The Beatles
3,3,5c42e9839fc5af27c4b76a25,The Beatles,"['G', 'Bm', 'Em', 'Em7', 'C', 'C6', 'C', 'C', ...",0,74,https://tabs.ultimate-guitar.com/tab/the_beatl...,A Day In The Life,Ver 4,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",A Day In The Life The Beatles
4,4,5c42eaba9fc5af27c4b76a2d,The Beatles,"['G', 'Bm', 'Em', 'Em7', 'C', 'C', 'G', 'F', '...",0,4,https://tabs.ultimate-guitar.com/tab/the_beatl...,A Day In The Life,Ver 5,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",A Day In The Life The Beatles


In [8]:
db=MongoConnect.mongoconnect()
one_document=db.Youtube_has_downloaded
list=one_document.find({'error':0})
download_list=[]
load=0
for i in list:
    load=load+1
    if(load % 100==0):print(load)
    download_list.append(i)

100


In [9]:
download_list=pd.DataFrame(download_list)
download_list.head()

,_id,error,song_detector_chord,song_name,song_yt_url
0,5c9c4dec9fc5af48a88b0c03,0,"[{'st': 0, 'et': 2.508, 'ochord': 'G'}, {'st':...",Got To Get You Into My Life The Beatles,https://www.youtube.com/watch?v=bxhhFOnXs2M
1,5c9c4e059fc5af48a88b0c05,0,"[{'st': 0, 'et': 0.906, 'ochord': 'F#:min'}, {...",Ill Be Back The Beatles,https://www.youtube.com/watch?v=2K9Ij6VmOD4
2,5c9c4e1b9fc5af48a88b0c07,0,"[{'st': 0, 'et': 0.557, 'ochord': 'G:min'}, {'...",A Taste Of Honey The Beatles,https://www.youtube.com/watch?v=PLyG_hkuLPk
3,5c9c4e329fc5af48a88b0c09,0,"[{'st': 0, 'et': 0.929, 'ochord': 'N'}, {'st':...",From Me To You The Beatles,https://www.youtube.com/watch?v=cJmXKnttMMQ
4,5c9c4e4e9fc5af48a88b0c0b,0,"[{'st': 0, 'et': 5.503, 'ochord': 'N'}, {'st':...",Oh Darling The Beatles,https://www.youtube.com/watch?v=NdGLEVKwHPU


In [12]:
join_right=pd.DataFrame(download_list[['song_name', 'song_detector_chord']])
song_train_example=pd.merge(song_train, join_right, on='song_name')
song_train_example

,Unnamed: 0,_id,artist,chord,error,rating,tab_href,tab_title,version,web_html,song_name,song_detector_chord
0,6,5c42ead99fc5af27c4b76a33,The Beatles,"['D7sus4', 'Fadd9/D', 'F6add9/A', 'F6add9/C', ...",0,735,https://tabs.ultimate-guitar.com/tab/the_beatl...,A Hard Days Night,Ver 1,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",A Hard Days Night The Beatles,"[{'st': 0, 'et': 1.834, 'ochord': 'N'}, {'st':..."
1,7,5c42eae39fc5af27c4b76a35,The Beatles,"['C7sus4', 'C', 'F', 'C', 'Bb', 'C', 'F', 'C',...",0,31,https://tabs.ultimate-guitar.com/tab/the_beatl...,A Hard Days Night,Ver 2,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",A Hard Days Night The Beatles,"[{'st': 0, 'et': 1.834, 'ochord': 'N'}, {'st':..."
2,8,5c42eaed9fc5af27c4b76a37,The Beatles,"['G7sus4', 'G', 'C', 'G', 'G', 'F', 'G', 'G', ...",0,184,https://tabs.ultimate-guitar.com/tab/the_beatl...,A Hard Days Night,Ver 3,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",A Hard Days Night The Beatles,"[{'st': 0, 'et': 1.834, 'ochord': 'N'}, {'st':..."
3,9,5c42eaf79fc5af27c4b76a39,The Beatles,"['G', 'C', 'G', 'F', 'G', 'C', 'G', 'F', 'G', ...",0,12,https://tabs.ultimate-guitar.com/tab/the_beatl...,A Hard Days Night,Ver 4,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",A Hard Days Night The Beatles,"[{'st': 0, 'et': 1.834, 'ochord': 'N'}, {'st':..."
4,10,5c42eb019fc5af27c4b76a3b,The Beatles,"['Gsus4', 'G', 'C', 'G', 'F', 'G', 'G', 'C', '...",0,25,https://tabs.ultimate-guitar.com/tab/the_beatl...,A Hard Days Night,Ver 5,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",A Hard Days Night The Beatles,"[{'st': 0, 'et': 1.834, 'ochord': 'N'}, {'st':..."
5,11,5c42eb0c9fc5af27c4b76a3d,The Beatles,"['Fadd9', 'G', 'Cadd9', 'G', 'Dm7', 'G', 'G', ...",0,51,https://tabs.ultimate-guitar.com/tab/the_beatl...,A Hard Days Night,Ver 6,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",A Hard Days Night The Beatles,"[{'st': 0, 'et': 1.834, 'ochord': 'N'}, {'st':..."
6,12,5c42eb169fc5af27c4b76a3f,The Beatles,"['A', 'D', 'A', 'G6', 'A', 'A', 'D', 'A', 'G6'...",0,3,https://tabs.ultimate-guitar.com/tab/the_beatl...,A Hard Days Night,Ver 7,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",A Hard Days Night The Beatles,"[{'st': 0, 'et': 1.834, 'ochord': 'N'}, {'st':..."
7,13,5c42eb209fc5af27c4b76a41,The Beatles,"['E', 'A', 'D', 'B', 'A7', 'G#m', 'C#m', 'E', ...",0,7,https://tabs.ultimate-guitar.com/tab/the_beatl...,A Hard Days Night,Ver 8,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",A Hard Days Night The Beatles,"[{'st': 0, 'et': 1.834, 'ochord': 'N'}, {'st':..."
8,14,5c42eb2a9fc5af27c4b76a43,The Beatles,"['G', 'C', 'D', 'F', 'Dm', 'Bm', 'Em', 'G', 'C...",0,35,https://tabs.ultimate-guitar.com/tab/the_beatl...,A Hard Days Night,Ver 9,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",A Hard Days Night The Beatles,"[{'st': 0, 'et': 1.834, 'ochord': 'N'}, {'st':..."
9,15,5c42eb349fc5af27c4b76a45,The Beatles,"['D', 'D', 'G7', 'D', 'A7', 'G7', 'A7', 'G7', ...",0,3,https://tabs.ultimate-guitar.com/tab/the_beatl...,A Shot Of Rhythm And Blues,Ver 1,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",A Shot Of Rhythm And Blues The Beatles,"[{'st': 0, 'et': 9.985, 'ochord': 'N'}, {'st':..."


In [30]:
song_train_example['song_chord']=song_train_example['song_detector_chord'].apply(retrie_song_chord)
song_train_example.head()

,Unnamed: 0,_id,artist,chord,error,rating,tab_href,tab_title,version,web_html,song_name,song_detector_chord,song_chord
0,6,5c42ead99fc5af27c4b76a33,The Beatles,"['D7sus4', 'Fadd9/D', 'F6add9/A', 'F6add9/C', ...",0,735,https://tabs.ultimate-guitar.com/tab/the_beatl...,A Hard Days Night,Ver 1,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",A Hard Days Night The Beatles,"[{'st': 0, 'et': 1.834, 'ochord': 'N'}, {'st':...","[N, C#, G, F, G, F, G, C, D, G, F, G, F, G, C,..."
1,7,5c42eae39fc5af27c4b76a35,The Beatles,"['C7sus4', 'C', 'F', 'C', 'Bb', 'C', 'F', 'C',...",0,31,https://tabs.ultimate-guitar.com/tab/the_beatl...,A Hard Days Night,Ver 2,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",A Hard Days Night The Beatles,"[{'st': 0, 'et': 1.834, 'ochord': 'N'}, {'st':...","[N, C#, G, F, G, F, G, C, D, G, F, G, F, G, C,..."
2,8,5c42eaed9fc5af27c4b76a37,The Beatles,"['G7sus4', 'G', 'C', 'G', 'G', 'F', 'G', 'G', ...",0,184,https://tabs.ultimate-guitar.com/tab/the_beatl...,A Hard Days Night,Ver 3,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",A Hard Days Night The Beatles,"[{'st': 0, 'et': 1.834, 'ochord': 'N'}, {'st':...","[N, C#, G, F, G, F, G, C, D, G, F, G, F, G, C,..."
3,9,5c42eaf79fc5af27c4b76a39,The Beatles,"['G', 'C', 'G', 'F', 'G', 'C', 'G', 'F', 'G', ...",0,12,https://tabs.ultimate-guitar.com/tab/the_beatl...,A Hard Days Night,Ver 4,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",A Hard Days Night The Beatles,"[{'st': 0, 'et': 1.834, 'ochord': 'N'}, {'st':...","[N, C#, G, F, G, F, G, C, D, G, F, G, F, G, C,..."
4,10,5c42eb019fc5af27c4b76a3b,The Beatles,"['Gsus4', 'G', 'C', 'G', 'F', 'G', 'G', 'C', '...",0,25,https://tabs.ultimate-guitar.com/tab/the_beatl...,A Hard Days Night,Ver 5,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",A Hard Days Night The Beatles,"[{'st': 0, 'et': 1.834, 'ochord': 'N'}, {'st':...","[N, C#, G, F, G, F, G, C, D, G, F, G, F, G, C,..."


In [14]:
def retrie_song_chord(song_detector_chord):
    song_chord=[]
    for i in song_detector_chord:
        song_chord.append(i['ochord'])
    return song_chord